In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time
import random
import json
import gc
from typing import Tuple, Optional, Dict
from functools import partial

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import nibabel as nib
from einops import rearrange
from scipy import ndimage
import wandb

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from research.data.natural_scenes import (
    NaturalScenesDataset
)
from research.models.components_2d import BlurConvTranspose2d
from research.models.fmri_decoders import VariationalDecoder, SpatialDecoder, SpatialDiscriminator, Decoder
from research.metrics.loss_functions import (
    EuclideanLoss, 
    EmbeddingClassifierLoss,
    ProbabalisticCrossEntropyLoss,
    VariationalLoss,
    CosineSimilarityLoss,
    EmbeddingDistributionLoss,
    ContrastiveDistanceLoss,
)
from research.experiments.nsd_decoding import NSDExperiment
from research.metrics.metrics import (
    cosine_similarity, 
    r2_score,
    pearsonr, 
    embedding_distance,
    cosine_distance,
    squared_euclidean_distance,
    contrastive_score,
    two_versus_two,
    smooth_euclidean_distance,
)

In [3]:
dataset_path = Path('D:\\Datasets\\NSD\\')
dataset = NaturalScenesDataset(dataset_path)

In [7]:
dataset.subjects['subj01']['betas'][100]

array([0, 0, 0, ..., 0, 0, 0], dtype=int16)

In [ ]:
dataset_params = dict(
    subject_name='subj01',
    model_name='DPT_Large',
    embedding_name='transformer.resblocks.0',
    encoder_name='fracridge',
    split_name='split-01',
    num_voxels=2500,
    normalize_X=True,
    normalize_Y=False,
)
train, val, test = dataset.load_data(**dataset_params)